In [1]:
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_auth = get_key('koray_data')

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

file_url = '/search/?type=FileFastq&status=uploading&status=upload failed&limit=all'
file_url = '/search/?type=FileReference&status=uploading&status=upload failed&limit=all'

file_url = '/search/?status=uploading&type=FileProcessed'
all_files = ff_utils.search_metadata(file_url, key=my_auth)
print(len(all_files))

1


In [2]:
run_md5 = input("Do you wanna run md5 if missing? (md5/none)")

printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            print("Deleted File", a_file)
            continue
    except:
        print(a_file)
        break
        
    if counter-printn > 100:
        print(counter)
        printn = counter

    if 'FileProcessed' in a_file['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket       
 
    # check if file is in s3
    head_info = my_s3_util.does_key_exist(a_file['upload_key'], my_bucket)
    if not head_info:
        print(a_file['accession'], a_file.get('aliases')[0], "does not have a file in S3")
        print()
        continue
        
    file_id = a_file['accession']
    status = a_file.get('status')
    attributions = get_attribution(a_file)
    
    # is there md5sum for gzip
    if not a_file.get('md5sum'):
        print(file_id,"does not have the md5sum calculated during upload")
    # Check workflows
    md5_status = 'did_not_run'
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    md5_report = get_wfr_out(a_file['uuid'], "md5", my_auth, md_qc = True, run=1)
    
    if md5_report['status'] == 'running':
        print(file_id, 'still running')
        continue 

    if status in ["uploading", "upload failed"] and md5_report['status'] != 'complete':
        # if not, shall we run it?
        if run_md5 == 'md5':
            print('md5 running for', file_id)
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
            print()
        # user does not want it to be run, so just report
        else:
            print('md5 run missing for', file_id)

    elif status in ["uploading", "upload failed"] and md5_status == 'complete':
        print("ERROR:",file_id ,"There is a successful md5 run but status not switched")
        # if not, shall we run it?
        if run_md5 == 'md5':
            print('md5 running for', file_id)
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
            print()
        
    elif md5_status != 'complete':
        if a_file.get('content_md5sum'):
            print('md5run not linked to file')
            if run_md5 == 'md5':
                print('md5 running for', file_id)
                inp_f = {'input_file':a_file['@id']}
                run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
                print()
        else:
            print(file_id, status, 'switched status file does not have md5 run or content_md5sum')
            # if not, shall we run it?
            if run_md5 == 'md5':
                print('md5 running for', file_id)
                inp_f = {'input_file':a_file['@id']}
                run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
                print()
    print()



Do you wanna run md5 if missing? (md5/none)md5
object 995ec5b2-536e-45c5-8449-f56615f91697/4DNFI8IETQEJ.bed.gz not found on bucket elasticbeanstalk-fourfront-webprod-wfoutput
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI8IETQEJ cremins-lab:Communities_genomewideHFFc68kbhg38merge_adjust2_PCA_0.25v2_2 does not have a file in S3



In [ ]:
### Check if md5 runs are in sync with file status
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')


md5_runs = '/search/?run_status=complete&status=in+review+by+lab&type=WorkflowRunAwsem&workflow.title=md5'
md5_hits = ff_utils.search_metadata(md5_runs , key=my_key)

print(len(md5_hits))

prob = 0
for a_run in md5_hits:
    f_status = a_run['input_files'][0]['value']['status']
    if f_status != a_run['status']:
        print(f_status, a_run['status'])
        prob += 1
    
print(prob)


patch = True
need_patch = 0
need_patch_run = 0
counter = 0

In [ ]:

counter = 0
for a_run in md5_hits:
    counter += 1
    if counter % 100 == 0:
        print()
        print(counter)
    f_status = a_run['input_files'][0]['value']['status']
    if f_status in ['archived', 'released', 'released to project']:
        print(f_status, a_run['status'])
        ff_utils.patch_metadata({"status": f_status}, obj_id=a_run['uuid'] ,key=my_key)
print(counter)
    
       